In [0]:
# Instructions

# Open this and the other notebook (BERTServiceForFFNNConversions.ipynb)

# make sure both notebooks are on GPU mode. Make sure they are both connected, the top right of the notebook should NOT read 'busy' or 'connecting'. 
#Note: These notebooks won't work in CPU or TPU mode (perhaps there's a workaround)

#Run the first line of code in this Notebook, do not run anything else. Bert As a Service is not compatible with TF 2.0, so do not intall TF2.0 yet. 

#Run all the other lines of BERTServiceForFFNNConversions.ipynb

#Wait 10 seconds after Bert as a service is running. The output of the last line will say something like "I:VENTILATOR:[__i:_ru:163]:all set, ready to serve request!"

#Run all other lines of this notebook. Some how installing TF 2.0 after bert service is running will not interfere with it. 

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import os

In [0]:
#DO NOT INSTALL TF 2.0 UNTIL BERT SERVICE IS RUNNING

In [4]:
import numpy as np
# install tf 2.0
from __future__ import absolute_import, division, print_function, unicode_literals

#DO NOT INSTALL TF 2.0 UNTIL BERT SERVICE IS RUNNING
!pip install tensorflow-gpu==2.0.0-alpha0
import tensorflow as tf
# tf.compat.v1.disable_v2_behavior()

print(tf.__version__)
import requests


    100% |████████████████████████████████| 332.1MB 47kB/s 
    100% |████████████████████████████████| 3.0MB 5.5MB/s 
    100% |████████████████████████████████| 419kB 6.8MB/s 
    100% |████████████████████████████████| 61kB 28.2MB/s 
2.0.0-alpha0


In [0]:
import os
from glob import glob
import numpy as np
from sklearn.model_selection import train_test_split

SEED = 42


def _float_list_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))


def _int64_list_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def create_generator_for_ffn(
        file_list,
        mode='train'):

    # file_list = glob(os.path.join(data_dir, '*.csv'))

    for full_file_path in file_list:
        # full_file_path = os.path.join(data_dir, file_name)
        if not os.path.exists(full_file_path):
            raise FileNotFoundError("File %s not found" % full_file_path)
        df = pd.read_csv(full_file_path, encoding='utf8')

        # so train test split
        if mode == 'train':
            df, _ = train_test_split(df, test_size=0.2, random_state=SEED)
        else:
            _, df = train_test_split(df, test_size=0.2, random_state=SEED)

        for _, row in df.iterrows():
            q_vectors = np.fromstring(row.question_bert.replace(
                '[[', '').replace(']]', ''), sep=' ')
            a_vectors = np.fromstring(row.answer_bert.replace(
                '[[', '').replace(']]', ''), sep=' ')
            vectors = np.stack([q_vectors, a_vectors], axis=0)
            if mode in ['train', 'eval']:
                print(vectors)                                      #added 
                yield vectors, 1
            else:
                print(vectors)                                        #added 
                yield vectors


def ffn_serialize_fn(features):
    features_tuple = {'features': _float_list_feature(
        features[0].flatten()), 'labels': _int64_feature(features[1])}
    example_proto = tf.train.Example(
        features=tf.train.Features(feature=features_tuple))
    return example_proto.SerializeToString()


def make_tfrecord(data_dir, generator_fn, serialize_fn, suffix='', **kwargs):
    """Function to make TF Records from csv files
    This function will take all csv files in data_dir, convert them
    to tf example and write to *_{suffix}_train/eval.tfrecord to data_dir.

    Arguments:
        data_dir {str} -- dir that has csv files and store tf record
        generator_fn {fn} -- A function that takes a list of filepath and yield the
        parsed recored from file.
        serialize_fn {fn} -- A function that takes output of generator fn and convert to tf example

    Keyword Arguments:
        suffix {str} -- suffix to add to tf record files (default: {''})
    """
    file_list = glob(os.path.join(data_dir, '*.csv'))
    train_tf_record_file_list = [
        f.replace('.csv', '_{0}_train.tfrecord'.format(suffix)) for f in file_list]
    test_tf_record_file_list = [
        f.replace('.csv', '_{0}_eval.tfrecord'.format(suffix)) for f in file_list]
    for full_file_path, train_tf_record_file_path, test_tf_record_file_path in zip(file_list, train_tf_record_file_list, test_tf_record_file_list):
        print('Converting file {0} to TF Record'.format(full_file_path))
        with tf.io.TFRecordWriter(train_tf_record_file_path) as writer:
            for features in generator_fn([full_file_path], mode='train', **kwargs):
                example = serialize_fn(features)
                writer.write(example)
        with tf.io.TFRecordWriter(test_tf_record_file_path) as writer:
            for features in generator_fn([full_file_path], mode='eval', **kwargs):
                example = serialize_fn(features)
                writer.write(example)


def create_dataset_for_ffn(
        data_dir,
        mode='train',
        hidden_size=768,
        shuffle_buffer=10000,
        prefetch=10000,
        batch_size=32):

    tfrecord_file_list = glob(os.path.join(
        data_dir, '*_FFN_{0}.tfrecord'.format((mode))))
    if not tfrecord_file_list:
        print('TF Record not found')
        make_tfrecord(
            data_dir, create_generator_for_ffn,
            ffn_serialize_fn, 'FFN')

    dataset = tf.data.TFRecordDataset(tfrecord_file_list)

    def _parse_ffn_example(example_proto):
        feature_description = {
            'features': tf.io.FixedLenFeature([2*768], tf.float32),
            'labels': tf.io.FixedLenFeature([], tf.int64, default_value=0),
        }
        feature_dict = tf.io.parse_single_example(
            example_proto, feature_description)
        return tf.reshape(feature_dict['features'], (2, 768)), feature_dict['labels']
    dataset = dataset.map(_parse_ffn_example)

    dataset = dataset.shuffle(shuffle_buffer)

    dataset = dataset.prefetch(prefetch)

    dataset = dataset.batch(batch_size)
    return dataset

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

import tensorflow as tf
import tensorflow.keras.backend as K


class FFN(tf.keras.layers.Layer):
    def __init__(
            self,
            hidden_size=768,                                                                #SG edit from 768 4-24-19
            dropout=0.2,
            residual=True,
            name='FFN',
            **kwargs):
        """Simple Dense wrapped with various layers
        """

        super(FFN, self).__init__(name=name, **kwargs)
        self.hidden_size = hidden_size
        self.dropout = dropout
        self.residual = residual
        self.ffn_layer = tf.keras.layers.Dense(
            units=hidden_size,
            use_bias=True
        )

    def call(self, inputs):
        ffn_embedding = self.ffn_layer(inputs)
        ffn_embedding = tf.keras.layers.ReLU()(ffn_embedding)
        if self.dropout > 0:
            ffn_embedding = tf.keras.layers.Dropout(
                self.dropout)(ffn_embedding)
#         ffn_embedding = self.ffn_layer(inputs)  #SG edit from 768 4-24-19
#         ffn_embedding = tf.keras.layers.ReLU()(ffn_embedding)  #SG edit from 768 4-24-19
#         if self.dropout > 0:  #SG edit from 768 4-24-19
#             ffn_embedding = tf.keras.layers.Dropout(  #SG edit from 768 4-24-19
#                 self.dropout)(ffn_embedding)  #SG edit from 768 4-24-19


        if self.residual:
            ffn_embedding += inputs
        return ffn_embedding


class MedicalQAModel(tf.keras.Model):
    def __init__(self, name=''):
        super(MedicalQAModel, self).__init__(name=name)
        self.q_ffn = FFN(name='QFFN', input_shape=(768,))
        self.a_ffn = FFN(name='AFFN', input_shape=(768,))

    def call(self, inputs):
#         print(inputs)
#         tf.print(inputs)
        q_bert_embedding, a_bert_embedding = tf.unstack(inputs, axis=1)
#         print(q_bert_embedding)
#         print(a_bert_embedding)
#         tf.print(q_bert_embedding)
#         tf.print(a_bert_embedding)
        q_embedding, a_embedding = self.q_ffn(
            q_bert_embedding), self.a_ffn(a_bert_embedding)
        return tf.stack([q_embedding, a_embedding], axis=1)


class BioBert(tf.keras.Model):
    def __init__(self, name=''):
        super(BioBert, self).__init__(name=name)

    def call(self, inputs):

        # inputs is dict with input features
        input_ids, input_masks, segment_ids = inputs
        # pass to bert
        # with shape of (batch_size/2*batch_size, max_seq_len, hidden_size)
        # TODO(Alex): Add true bert model
        # Input: input_ids, input_masks, segment_ids all with shape (None, max_seq_len)
        # Output: a tensor with shape (None, max_seq_len, hidden_size)
        fake_bert_output = tf.expand_dims(tf.ones_like(
            input_ids, dtype=tf.float32), axis=-1)*tf.ones([1, 1, 768], dtype=tf.float32)
        max_seq_length = tf.shape(fake_bert_output)[-2]
        hidden_size = tf.shape(fake_bert_output)[-1]

        bert_output = tf.reshape(
            fake_bert_output, (-1, 2, max_seq_length, hidden_size))
        return bert_output


class MedicalQAModelwithBert(tf.keras.Model):
    def __init__(
            self,
            hidden_size=768,
            dropout=0.2,
            residual=True,
            activation=tf.keras.layers.ReLU(),
            name=''):
        super(MedicalQAModelwithBert, self).__init__(name=name)
        self.biobert = BioBert()
        self.q_ffn_layer = FFN(
            hidden_size=hidden_size,
            dropout=dropout,
            residual=residual,
            activation=activation)
        self.a_ffn_layer = FFN(
            hidden_size=hidden_size,
            dropout=dropout,
            residual=residual,
            activation=activation)

    def _avg_across_token(self, tensor):
        if tensor is not None:
            tensor = tf.reduce_mean(tensor, axis=1)
        return tensor

    def call(self, inputs):

        q_bert_embedding, a_bert_embedding = self.biobert(inputs)

        # according to USE, the DAN network average embedding across tokens
        q_bert_embedding = self._avg_across_token(q_bert_embedding)
        a_bert_embedding = self._avg_across_token(a_bert_embedding)

        q_embedding = self.q_ffn_layer(q_bert_embedding)
        a_embedding = self.a_ffn_layer(a_bert_embedding)

        return tf.stack([q_embedding, a_embedding], axis=1)

      
      
# def qa_pair_cross_entropy_loss(y_true, y_pred):
#     y_true = tf.eye(tf.shape(y_pred)[0])
#     q_embedding, a_embedding = tf.unstack(y_pred, axis=1)
#     similarity_matrix = tf.matmul(
#         q_embedding, a_embedding, transpose_b=True)
#     similarity_matrix_logits = tf.math.sigmoid(similarity_matrix)
#     return tf.keras.losses.categorical_crossentropy(y_true, similarity_matrix_logits, from_logits=True)

def qa_pair_cross_entropy_loss(y_true, y_pred):
    y_true = tf.eye(tf.shape(y_pred)[0])
    q_embedding, a_embedding = tf.unstack(y_pred, axis=1)
    similarity_matrix = tf.matmul(
        a = q_embedding, b = a_embedding, transpose_b=True)
    similarity_matrix_softmaxed = tf.nn.softmax(similarity_matrix)
    K.print_tensor(similarity_matrix_softmaxed, message="similarity_matrix_softmaxed is: ")
    return tf.keras.losses.categorical_crossentropy(y_true, similarity_matrix_softmaxed, from_logits=False)



In [0]:
# training config
batch_size = 64
num_epochs=35
learning_rate=0.0001
validation_split=0.2
shuffle_buffer=50000
prefetch=50000

#Gdrive link to tfrecords
#https://drive.google.com/open?id=1wRc1jtl5Q0objpfualNFwpg4H575tmks
#Place the files in your own Google drive in a folder called 'mqa_tf_record'
data_path='/content/gdrive/My Drive/mqa_tf_record'

#Gdrive link to model
#https://drive.google.com/open?id=1TxR8UBzoBtlewO3lt5efLQgGP_3BRPZm
#Place the files in your own Google drive in a folder called 'mqa_models'
model_path = '/content/gdrive/My Drive/mqa_models/ffn_model_cross_entropy'

In [0]:
  d = create_dataset_for_ffn(
      data_path, batch_size=batch_size, shuffle_buffer=shuffle_buffer, prefetch=prefetch)
  eval_d = create_dataset_for_ffn(
      data_path, batch_size=batch_size, mode='eval')
  medical_qa_model = MedicalQAModel()
  optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
  medical_qa_model.compile(
      optimizer=optimizer, loss=qa_pair_cross_entropy_loss)

  epochs = num_epochs
  loss_metric = tf.keras.metrics.Mean()

#   history = medical_qa_model.fit(d, epochs=epochs, validation_data=eval_d)
#   history = medical_qa_model.train_on_batch(d)

In [0]:
model_path2 = '/content/gdrive/My Drive/mqa_models/ffn_model_cross_entropy.ckpt'

checkpoint = tf.keras.callbacks.ModelCheckpoint(model_path2, monitor='loss', verbose=1, save_best_only=True)

In [10]:
checkpoint_dir = os.path.dirname(model_path2)
print(checkpoint_dir)

/content/gdrive/My Drive/mqa_models


In [11]:
medical_qa_model.load_weights(model_path2)

In [30]:
#This is optional, it's just to make sure weights are loaded correctly. Loss should start below 2 for batch size 64

history = medical_qa_model.fit(d, epochs=2, validation_data=eval_d )

Epoch 1/2


W0427 16:37:29.969590 140303120930688 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/backend.py:2924: Print (from tensorflow.python.ops.logging_ops) is deprecated and will be removed after 2018-08-20.
Instructions for updating:
Use tf.print instead of tf.Print. Note that tf.print returns a no-output operator that directly prints the output. Outside of defuns or eager mode, this operator will not be executed unless it is directly specified in session.run or used as a control dependency for other operators. This is only a concern in graph mode. Below is an example of how to ensure tf.print executes in graph mode:
```python
    sess = tf.Session()
    with sess.as_default():
        tensor = tf.range(10)
        print_op = tf.print(tensor)
        with tf.control_dependencies([print_op]):
          out = tf.add(tensor, tensor)
        sess.run(out)
    ```
Additionally, to use tf.print in python 2.7, users must make sure to import
the following:

 

3349/3349 [==============================] - 52s 16ms/step - loss: 1.3162 - val_loss: 0.0000e+00
Epoch 2/2
3349/3349 [==============================] - 27s 8ms/step - loss: 1.2721 - val_loss: 1.9777


In [0]:
from bert_serving.client import BertClient

In [0]:
bc = BertClient()

In [0]:
def QgetEmbeds(x):
    bertEmbed= bc.encode([x])
    textinput = tf.constant([bertEmbed[0], bertEmbed[0]])
    textinputF = tf.dtypes.cast(textinput , dtype=tf.float32) 
    textinputE = tf.expand_dims(textinputF, 0)
    q_embedding, a_embedding = tf.unstack(medical_qa_model(textinputE), axis=1)
    return q_embedding.numpy()

In [0]:
inputtext = "Is this schizophrenia? How can I get appropriate help for it in the middle of nowhere? Age: 18 Sex: Male Height: 5 10 Weight: 158 lb Race: Black Duration of complaint: 3 years Location (Geographic and on body): Psyche Any existing relevant medical issues (if any): Trauma from child abuse and molestation Current medications (if any): None Include a photo if relevant (skin condition for example)I think I might have schizophrenia. I've felt suicidal for a long time, and I started hearing voices about three years ago that come and go. Sometimes they say things that make no sense but have a sort of poetic rhythm. Example: The crime of the nine crushes the dime of the lime. I've been reading Freud and other investigators of the subconscious trying to find the meaning of the things I hear, but I haven't come across anything conclusive. The voices can also be nasty or say things that are untrue or paranoid. Sometimes they straight up call me ugly and stupid and say I should kill myself. Sometimes when I see a white person, a voice says they hate black people and want to kill me. I know it's just my imagination, but I get suspicious and keep my guard up around them. I really don't know if I have delusions. From what I've read, they're hard to recognize for the individual. I don't trust anything the voices say, but I still feel depressed and suicidal due to life circumstances. I live with my mom and her boyfriend, and they constantly fight and hit each other. My mom is 90% of the problem because she has hair trigger emotions and gets very mad over nothing. My whole life feels extremely hopeless, and I sleep excessively and sob all the time. I had two friends who both stopped talking to me and moved away. We live in the middle of nowhere and the nearest mental hospital is an hour and a half away. I'd have to take her car, and if it's bad enough that they want to strap me to a bed as they often do with suicidal people, then I wouldn't be able to bring her car back for a while. There is a clinic in town that my mom won't let me go to because the doctors all hate black people. She said they misdiagnosed her breast cancer and tried to implant a chip inside her because they want to keep track of all the black people. She said they were taking a blood sample and injected a neurotoxin. This sounds very paranoid to me, but regardless she's the one with the health insurance who gets to decide where I go and what I do. She had to fly to Rochester, MN for cancer treatment because nobody in our area would help her when she had a breast lump that required a mastectomy."

In [0]:
testtt = QgetEmbeds(inputtext)

In [29]:
print(testtt)

[[-7.41489232e-02 -2.93513060e-01  7.94782341e-01  3.73879313e-01
  -1.83415543e-02 -1.18541509e-01  1.46455944e-01 -1.50270730e-01
   8.95201683e-01  4.57437724e-01  4.25101250e-01  6.55673265e-01
   9.01300162e-02  5.38662851e-01  2.21652165e-02 -1.02907240e-01
   1.15937382e-01  3.38005483e-01 -3.31434935e-01  8.24319571e-02
   1.27929258e+00  2.15683386e-01  5.31941831e-01 -1.56582832e-01
  -1.59840472e-02  2.44420439e-01  3.93882424e-01  9.94355857e-01
   1.18196058e+00  2.55656362e-01 -1.95982307e-02  5.19520521e-01
  -2.01406032e-01 -2.40532070e-01  2.62885481e-01  1.97201937e-01
  -1.45242333e-01  2.97877640e-01 -2.97187492e-02  6.45119548e-01
   1.00307986e-01  2.55139470e-01  1.61336005e-01  2.63374090e-01
   3.20117831e-01  7.69968152e-01  6.99028373e-02 -1.43000603e-01
  -5.75600043e-02 -1.89701223e-03  3.89344215e-01  7.32022464e-01
   9.77720976e-01 -5.88473082e-02  1.02633631e+00 -1.07916273e-01
  -8.85868371e-02  4.19590503e-01 -3.18220615e-01  7.32022703e-01
   1.23360